# Polya Gamma Sampler

In [10]:
import numpy as np

class PolyaGammaSampler:

    def __init__(self):
        pass
    
    def sample_scalar(self, b, c, truncation=100):
        total = 0.0
        for k in range(1, truncation + 1):
            term_1 = np.random.gamma(b, 1.0)
            term_2 = (k - 0.5)**2
            term_3 = (c / (2 * np.pi))**2
            total += term_1 / (term_2 + term_3)
        
        return 0.5 * total / (np.pi**2)

    def sample(self, b_array, c_array, truncation=100):
        b_array = np.asarray(b_array)
        c_array = np.asarray(c_array)
        N = b_array.shape[0]
        omega = np.zeros(N)
        
        for n in range(N):
            omega[n] = self.sample_scalar(b_array[n], c_array[n], truncation=truncation)
        
        return omega


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import nbinom
from tqdm import trange

# 1. Generate data
N = 200
X = np.hstack([np.ones((N, 1)), np.random.randn(N, 2)])
beta_true = np.array([0.5, 1.0, -1.5])
r_true = 10.0

eta = X @ beta_true
mu = np.exp(eta)
p = r_true / (r_true + mu)
y = nbinom.rvs(r_true, p)

 92%|█████████▏| 919/1000 [00:27<00:02, 32.78it/s]

In [ ]:
class PolyaGammaSampler:

    def __init__(self):
        pass
    
    def sample_scalar(self, b, c, truncation=100):
        total = 0.0
        for k in range(1, truncation + 1):
            term_1 = np.random.gamma(b, 1.0)
            term_2 = (k - 0.5)**2
            term_3 = (c / (2 * np.pi))**2
            total += term_1 / (term_2 + term_3)
        
        return 0.5 * total / (np.pi**2)

    def sample(self, b_array, c_array, truncation=100):
        b_array = np.asarray(b_array)
        c_array = np.asarray(c_array)
        N = b_array.shape[0]
        omega = np.zeros(N)
        
        for n in range(N):
            omega[n] = self.sample_scalar(b_array[n], c_array[n], truncation=truncation)
        
        return omega

# 3. Gibbs sampler setup
pg = PolyaGammaSampler()

In [ ]:
# Prior: beta ~ N(0, 100 I)
Sigma0_inv = np.eye(3) * 0.01
mu0 = np.zeros(3)

num_iters = 1000
beta_samples = []

# Initialize
beta = np.zeros(3)

# 4. Gibbs sampling loop
for it in trange(num_iters):
    # Step 1: Sample PG variables
    eta = X @ beta
    b_vec = y + r_true
    c_vec = eta
    omega = pg.sample(b_vec, c_vec, truncation=200)
    
    # Step 2: Sample beta
    Omega = np.diag(omega)
    V_inv = X.T @ Omega @ X + Sigma0_inv
    V = np.linalg.inv(V_inv)
    
    m = V @ (X.T @ ((y - r_true) / 2) + Sigma0_inv @ mu0)
    
    beta = np.random.multivariate_normal(m, V)
    beta_samples.append(beta)

# Convert to array
beta_samples = np.array(beta_samples)

# 5. Plot results
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for d in range(3):
    axes[d].plot(beta_samples[:, d])
    axes[d].axhline(beta_true[d], color='red', linestyle='--')
    axes[d].set_title(f'beta_{d} trace')
plt.show()

# 6. Posterior means
print("Posterior means:")
print(beta_samples.mean(axis=0))
print("True beta:")
print(beta_true)
